# All cells
## Xiaonan Wang
## 17July2024

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
#from matplotlib_venn import venn2
#from matplotlib_venn import venn3
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
from functools import reduce
import seaborn as sns

cmap = LinearSegmentedColormap.from_list(name='gene_cmap', colors=['lightgrey', 'thistle', 'red', 'darkred']) 

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80, color_map='viridis')

In [ ]:
plt.rcParams["figure.figsize"] = (5,4.5)

In [ ]:
def expPlotFun(adata, group = 'Study', color='louvain_rd10', vmin=None, vmax=None, size=50):
    if vmax is None:
        if color in adata.raw.var_names:
            vmax = np.max(adata.raw[:,color].X)
    if vmin is None:
        vmin = 0
        
    gs = adata.obs[group].cat.categories
    
    ng = len(gs)+1
    fig,ax = plt.subplots(1,ng, figsize=(5*ng,4.5), sharex=True, sharey=True)
    
    ax1=sc.pl.umap(adata ,ax=ax[0], legend_loc="on data", color=color, show=False, size=size, color_map=cmap, vmin=vmin, vmax=vmax, legend_fontsize=15)
    for i in range(len(gs)):
        ax2=sc.pl.umap(adata[adata.obs[group]==gs[i],:], title=gs[i],ax=ax[i+1], legend_loc="on data", vmin=vmin, vmax=vmax, color=color, show=False, size=size, color_map=cmap, legend_fontsize=15)
    fig.tight_layout()

In [ ]:
def barplot_fun(adata, con1, con2, plot=True):
    t1 = pd.crosstab(adata.obs[con1], adata.obs[con2],dropna =False)
    t1 = t1.div(t1.sum(axis=0),axis=1)*10000
    t1_per = t1.div(t1.sum(axis=1),axis=0)*100
    
    if (plot==True):
        fig, ax = plt.subplots(nrows=1, ncols=2,figsize=(12,4))
        t1.plot.bar(edgecolor="black", ax=ax[0])
        ax[0].legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
        ax[0].set_ylabel('normalised # of cells')
        ax[0].grid()
        t1_per.plot.bar(stacked=True, edgecolor="black", ax=ax[1])
        ax[1].legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
        ax[1].set_ylabel('% in each cluster')
        ax[1].grid()
        fig.tight_layout()
    return t1_per

In [ ]:
adata = sc.read('/home/xw251/rds/rds-bg200-hphi-gottgens/users/xw251/From_lila_single_cell/Zebrafish/New_Batch_Analysis_v2/write/Final_allcells_norm_afterQC.h5ad')

## Read in Counts

In [ ]:
path = '/servers/lila-single-cell/Xiaonan/Zebrafish/NewData/181011_K00198_0353_BH2GVLBBXY/REX/WTCHG_587680_45/outs/raw_gene_bc_matrices/GRCz11/'
adata45 = sc.read(path + 'matrix.mtx', cache=True).T  # transpose the data
adata45.var_names = pd.read_csv(path + 'genes.tsv', header=None, sep='\t')[1]
adata45.obs_names = pd.read_csv(path + 'barcodes.tsv', header=None)[0]
adata45.var_names_make_unique()
adata45.obs['Study'] = ["WT"]*len(adata45.obs_names)
path = '/servers/lila-single-cell/Xiaonan/Zebrafish/NewData/181011_K00198_0353_BH2GVLBBXY/REX/WTCHG_587680_46/outs/raw_gene_bc_matrices/GRCz11/'
adata46 = sc.read(path + 'matrix.mtx', cache=True).T  # transpose the data
adata46.var_names = pd.read_csv(path + 'genes.tsv', header=None, sep='\t')[1]
adata46.obs_names = pd.read_csv(path + 'barcodes.tsv', header=None)[0]
adata46.var_names_make_unique()
adata46.obs['Study'] = ["Injured"]*len(adata46.obs_names)
path = '/servers/lila-single-cell/Xiaonan/Zebrafish/NewData/181011_K00198_0353_BH2GVLBBXY/REX/WTCHG_587680_47/outs/raw_gene_bc_matrices/GRCz11/'
adata47 = sc.read(path + 'matrix.mtx', cache=True).T  # transpose the data
adata47.var_names = pd.read_csv(path + 'genes.tsv', header=None, sep='\t')[1]
adata47.obs_names = pd.read_csv(path + 'barcodes.tsv', header=None)[0]
adata47.var_names_make_unique()
adata47.obs['Study'] = ["Injured_Runx1_KO"]*len(adata47.obs_names)

In [ ]:
adata = adata45.concatenate(adata46, adata47)
print(adata.shape)

In [ ]:
adata.obs['Study'] = adata.obs['Study'].astype('category')
adata.obs['Study'] = adata.obs['Study'].cat.reorder_categories(['WT', 'Injured', 'Injured_Runx1_KO'])

In [ ]:
adata.obs['n_genes_total'] = np.sum(adata.X>0, axis=1)
adata.obs['n_counts_total'] = adata.X.sum(axis=1).A1

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
sns.kdeplot(adata.obs['n_genes_total'],bw = 0.05, ax=ax[0])
ax[0].axvline(x=100, color='red')
sns.kdeplot(adata.obs['n_counts_total'],bw =0.05, ax=ax[1])
ax=ax[1].axvline(x=2000, color='red')
plt.tight_layout()

In [ ]:
sc.pp.filter_cells(adata, min_genes=100)
print(adata.shape)

In [ ]:
adata.obs['Study'].value_counts()

## Doublet Removal

The code here will not run as scrublet involves random sampling. Thus, the results will be similar but not the same as what we used.

In [ ]:
# import scrublet as scr
# db_scores = []
# predicted_db = []
# Study_unique = ['WT', 'Injured', 'Injured_Runx1_KO']
# Cell_index = np.array([])
# for i in Study_unique:
#     scrub = scr.Scrublet(adata[adata.obs['Study']==i,:].X)
#     doublet_scores, predicted_doublets = scrub.scrub_doublets()
#     db_scores.append(doublet_scores)
#     predicted_db.append(predicted_doublets)
#     Cell_index = np.append(Cell_index, adata[adata.obs['Study']==i,:].obs_names)
    
# db_scores_all = np.concatenate(db_scores)
# adata = adata[Cell_index,:]
# adata.obs['doublet_scores'] = db_scores_all

# fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(14, 4))
# thres = [0.3, 0.27, 0.38]
# for i in range(len(Study_unique)):
#     pd.DataFrame(db_scores[i]).plot.hist(bins=100, ax=ax[i])
#     ax[i].axvline(x=thres[i], color='b')
#     ax[i].set_title(Study_unique[i])
#     ax[i].get_legend().remove()
# plt.tight_layout()

# Cells_passed = np.array([])
# for i in range(len(Study_unique)):
#     newD = adata[adata.obs['Study']==Study_unique[i],:].obs_names
#     cells = newD[db_scores[i] < thres[i]]
#     print(Study_unique[i]+':'+str(len(cells)))
#     Cells_passed = np.append(Cells_passed, cells)

In [ ]:
# read in the previous saved scrublet results.
Cells_passed = np.genfromtxt('/servers/lila-single-cell/Xiaonan/Zebrafish/New_Batch_Analysis1/passed_CN.txt', dtype=str)

In [ ]:
adata = adata[Cells_passed, :]

In [ ]:
print(adata.shape)

In [ ]:
adata.obs['Study'].value_counts()

## Further QC

In [ ]:
sc.pp.filter_genes(adata, min_counts=1)

In [ ]:
print(adata.shape)

In [ ]:
mito_genes = [name for name in adata.var_names if name.startswith('mt-')]
# for each cell compute fraction of counts in mito genes vs. all genes
# the `.A1` is only necessary, as X is sparse - it transform to a dense array after summing
adata.obs['percent_mito'] = np.sum(
    adata[:, mito_genes].X, axis=1).A1 / np.sum(adata.X, axis=1).A1
# add the total counts per cell as observations-annotation to adata
adata.obs['n_counts'] = adata.X.sum(axis=1).A1

In [ ]:
sc.pl.violin(adata, ['n_genes', 'n_counts', 'percent_mito'],
             jitter=0.4, multi_panel=True)

Temporarily store the raw data as this is needed for quatifying marker genes.

In [ ]:
adata.raw = sc.pp.log1p(adata, copy=True)

## Find highly variable genes

In [ ]:
sc.pp.normalize_per_cell(adata, counts_per_cell_after=10000)

In [ ]:
filter_result = sc.pp.filter_genes_dispersion(
    adata.X, min_mean=0.0125, max_mean=4, min_disp=0.5)
print(sum(filter_result.gene_subset))
sc.pl.filter_genes_dispersion(filter_result)

In [ ]:
adata = adata[:, filter_result.gene_subset].copy()

In [ ]:
sc.pp.log1p(adata)

In [ ]:
sc.pp.regress_out(adata, ['n_counts', 'percent_mito'])

In [ ]:
sc.pp.scale(adata)

## Add in the vector information

The raw reads were aligned with the annotation with vector information involved and the vector informaiton was extracted

In [ ]:
path = '/servers/lila-single-cell/Xiaonan/Zebrafish/NewData/Cellranger_run1/587680_45-REX/outs/raw_gene_bc_matrices/Zebrafish_mCherry_Citrine/'
markerD45 = sc.read(path + 'matrix.mtx', cache=True).T  # transpose the data
markerD45.var_names = pd.read_csv(path + 'genes.tsv', header=None, sep='\t')[1]
markerD45.obs_names = pd.read_csv(path + 'barcodes.tsv', header=None)[0]
markerD45.var_names_make_unique()
path = '/servers/lila-single-cell/Xiaonan/Zebrafish/NewData/Cellranger_run1/587680_46-REX/outs/raw_gene_bc_matrices/Zebrafish_mCherry_Citrine/'
markerD46 = sc.read(path + 'matrix.mtx', cache=True).T  # transpose the data
markerD46.var_names = pd.read_csv(path + 'genes.tsv', header=None, sep='\t')[1]
markerD46.obs_names = pd.read_csv(path + 'barcodes.tsv', header=None)[0]
markerD46.var_names_make_unique()
path = '/servers/lila-single-cell/Xiaonan/Zebrafish/NewData/Cellranger_run1/587680_47-REX/outs/raw_gene_bc_matrices/Zebrafish_mCherry_Citrine/'
markerD47 = sc.read(path + 'matrix.mtx', cache=True).T  # transpose the data
markerD47.var_names = pd.read_csv(path + 'genes.tsv', header=None, sep='\t')[1]
markerD47.obs_names = pd.read_csv(path + 'barcodes.tsv', header=None)[0]
markerD47.var_names_make_unique()

In [ ]:
markerD = markerD45.concatenate(markerD46, markerD47)
marker_index = [x for x in markerD.var_names if 'mCherry' in x or 'Citrine' in x ]
markerD = markerD[:, marker_index]
print(markerD.X.shape)
print(markerD.var_names)

In [ ]:
mchr = markerD[markerD[:,'mCherry'].X>0,:].obs_names
mchr_pb = markerD[markerD[:,'mCherry-plasmid-backbone'].X>0,:].obs_names
mchr_pa = markerD[markerD[:,'mCherry-polyA'].X>0,:].obs_names
cit = markerD[markerD[:,'Citrine'].X>0,:].obs_names
cit_pa = markerD[markerD[:,'Citrine-polyA'].X>0,:].obs_names
cit_pb = markerD[markerD[:,'Citrine-Remaining'].X>0,:].obs_names

In [ ]:
mchr_overlap = reduce(np.union1d, (mchr,mchr_pb,mchr_pa))
cit_overlap = reduce(np.union1d, (cit,cit_pa,cit_pb))
print('mCherry cells: ', str(len(mchr_overlap)))
print('Citrine cells: ', str(len(cit_overlap)))

Now we define cit, mChr, cit_mChr, kdrl, runx1, kdrl_mchr, runx1_cit and eventually DBpos cells.

In [ ]:
adata.obs['cit'] = np.in1d(adata.obs_names, cit_overlap)*1
adata.obs['mChr'] = np.in1d(adata.obs_names, mchr_overlap)*1
adata.obs['cit_mChr'] = np.in1d(adata.obs_names, np.intersect1d(cit_overlap, mchr_overlap))*1
adata.obs['kdrl_exp'] = (adata.raw[:,'kdrl'].X > 0)*1
adata.obs['runx1_exp'] = (adata.raw[:,'runx1'].X > 0)*1
adata.obs['kdrl_mchr'] = ((adata.obs['kdrl_exp']==1) | (adata.obs['mChr']==1))*1
adata.obs['runx1_cit'] = ((adata.obs['runx1_exp']==1) | (adata.obs['cit']==1))*1
adata.obs['DBpos'] = ((adata.obs['kdrl_mchr']==1) & (adata.obs['runx1_cit']==1))*1

In [ ]:
print(adata.obs_keys())

In [ ]:
keys2change = adata.obs_keys()[7:16]

In [ ]:
print(keys2change)

In [ ]:
for i in keys2change:
    if 'DBpos' in i:
        newK = ['Double+', 'Double-']
    else:
        newK = [i+'+', i+'-']
    newObs = [newK[0] if x==1 else newK[1] for x in adata.obs[i]]
    adata.obs[i] = newObs
    print(adata.obs[i].value_counts())
    adata.obs[i] = adata.obs[i].astype('category')

## Dimensionality Reduction

In [ ]:
sc.tl.pca(adata)

In [ ]:
#np.savetxt('/servers/lila-single-cell/Xiaonan/Zebrafish/New_Batch_Analysis_v2/PCA_co_all.txt', adata.obsm['X_pca'], delimiter='\t')

In [ ]:
adata.obsm['X_pca'] = np.genfromtxt('/servers/lila-single-cell/Xiaonan/Zebrafish/New_Batch_Analysis_v2/PCA_co_all.txt', delimiter='\t')

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True)

In [ ]:
sc.tl.tsne(adata)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10)

In [ ]:
sc.tl.umap(adata)

In [ ]:
#np.savetxt('/servers/lila-single-cell/Xiaonan/Zebrafish/New_Batch_Analysis_v2/UMAP_co_all.txt', adata.obsm['X_umap'], delimiter='\t')

In [ ]:
adata.obsm['X_umap'] = np.genfromtxt('/servers/lila-single-cell/Xiaonan/Zebrafish/New_Batch_Analysis_v2/UMAP_co_all.txt', delimiter='\t')

In [ ]:
sc.tl.draw_graph(adata, maxiter=100)

## Louvain clustering

In [ ]:
sc.tl.louvain(adata, resolution = 1, random_state=0,key_added='louvain_rd10')

In [ ]:
sc.pl.umap(adata, color=['louvain_rd10'], legend_loc="on data", size=50, color_map=cmap)

In [ ]:
Lold = adata.obs['louvain_rd10'].astype('category')
Lold_lv = Lold.cat.categories
print(Lold_lv)

In [ ]:
Lnew_lv = ['3', '25', '19', '2', '8', '4', '1', '24', '15', '23', '7', '0', '17',
       '22', '21', '10', '13', '11', '26', '14', '5', '18', '20', '12', '16',
       '9', '6']
adata.obs['louvain_rd10'] = Lold.cat.rename_categories(Lnew_lv)
adata.obs['louvain_rd10'] = adata.obs['louvain_rd10'].cat.reorder_categories(Lold_lv)

In [ ]:
lv = ['0', '1', '2', '3', '4', '5', '6', 
      '7', '8', '9', '10', 
      '11', 
      '12','13',
      '14', '15','16', 
      '17', '18', 
      '19', '20', '21', '22', '23', 
      '24','25', 
      '26'
]

In [ ]:
adata.uns['louvain_rd10_colors'] = [
    '#0B5345', '#15AB0F', '#63D95E', '#1D8348','#4BB80C', '#52BE80', '#1EF861', 
    '#D4E95E', '#8CA11C', '#7B8003', '#444604',
    '#5175FB', 
    '#58DDF5', '#2190A4', 
    '#A42DE9', '#7015EB', '#ABB7FF',
    '#F22BA2', '#EC7DBF',
    '#CC3D6E', '#FCC176', '#FFADAB', '#A93226', '#FD1901',
    '#FF5733', '#ff9900',
    '#C0C0C0'    
]

In [ ]:
sc.pl.umap(adata, color='louvain_rd10', legend_loc="on data", size=50, color_map=cmap)

## Further analysis

Calculate proportion of each cluster in each sample.

In [ ]:
barplot_fun(adata, 'louvain_rd10', 'Study')

In [ ]:
adata.write('/servers/lila-single-cell/Xiaonan/Zebrafish/New_Batch_Analysis_v2/write/Final_allcells_raw_afterQC.h5ad')

Separated by samples.

In [ ]:
expPlotFun(adata, group='Study', color='louvain_rd10')

Separated by kdrl/mchr.

In [ ]:
expPlotFun(adata, group='kdrl_mchr', color='louvain_rd10')

Separated by runx1/cit.

In [ ]:
expPlotFun(adata, group='runx1_cit', color='louvain_rd10')

Separated by DBpos.

In [ ]:
expPlotFun(adata, group='DBpos', color='louvain_rd10')

## Find Marker Genes

In [ ]:
sc.tl.rank_genes_groups(adata, 'louvain_rd10', n_genes=adata.raw.X.shape[1], key_added='DE_cluster')

In [ ]:
adata = sc.read('./write/Final_allcells_norm_afterQC.h5ad')

In [ ]:
cl_unique = np.unique(adata.obs['louvain_rd10'])
for cl in cl_unique:
    print('Cluster Number: '+str(cl))
    cl = str(cl)
    DFprint = np.vstack([adata.uns['DE_cluster']['names'][cl],adata.uns['DE_cluster']['scores'][cl],adata.uns['DE_cluster']['logfoldchanges'][cl],adata.uns['DE_cluster']['pvals'][cl],adata.uns['DE_cluster']['pvals_adj'][cl]]).T
    DF = pd.DataFrame(DFprint, columns=['names', 'scores', 'logfoldchanges', 'pvals','pvals_adj'])
    DF.to_csv('./DEres/louvain_all/c'+str(cl)+'.csv')

In [ ]:
adata.obs['Study'].value_counts()

Plot key marker genes for each cluster.

In [ ]:
geneIDs = ['foxc1a','flt1', 'kdrl', #0-6 endothelial/endocardial cells
           'epcam', 'txn', #7 epitheial-like cells
           'tcf21','tbx18', 'col1a1a', #8 epicardial cells/fibroblasts
           'angptl7', #9 valve cells
           'tagln', #10 myofroblasts/smooth muscle
           'myl7','nkx2.5', #11 cardiomyocytes
           'fgfbp2b','cnmd', #12 fibrocytes
           'lect2l','lyz', #13 neutrophils
           'cd74a','cd9b', #14-16 monocyte derived macrophages
           'ccr9a','sla2', #17 lymophocytes/naive T-cells
           'il4','il13', #18 naive T-cells
           'gata2b','myb', # 19
           'stmn1a', 'cdk1', #20 cellular division of hematopoietic cells
           'lmo2','tal1', #19-21 HSCs
           'hbba1','hbaa1', #22-23 erythrocyts
           'apln','itga2b'#,  #24-25 thrombocytes          
          ]

In [ ]:
sc.pl.umap(adata, color=geneIDs, legend_loc="on data", size=50, color_map=cmap)

In [ ]:
adata.write('/servers/lila-single-cell/Xiaonan/Zebrafish/New_Batch_Analysis_v2/write/Final_allcells_norm_afterQC.h5ad')

# Rshiny frontpage plot

In [ ]:
adata = sc.read('./write/Final_allcells_norm_afterQC.h5ad')

In [ ]:
ct = {
    '0': 'EC',
    '1': 'EC',
    '2': 'EC',
    '3': 'EC',
    '4': 'EC',
    '5': 'EC',
    '6': 'EC',
    '7': 'EPI',
    '8': 'Epicardial/\nFibro',
    '9': 'VC',
    '10': 'MyoFibro/\nSMC',
    '11': 'CM',
    '12': 'F',
    '13': 'Neutro',
    '14': 'DC',
    '15': 'Mono/\nMacro',
    '16': 'Mono',
    '17': 'Lympho',
    '18': 'Maive T',
    '19': 'HSPC',
    '20': 'HSPC',
    '21': 'HSPC',
    '22': 'Erythro',
    '23': 'Erythro',
    '24': 'Thrombo',
    '25': 'Thrombo',
    '26': 'Dying'
}

In [ ]:
adata.obs['celltype'] = [ct[x] for x in adata.obs['louvain_rd10']]

In [ ]:
mean_co = {}
for i in np.unique(adata.obs['celltype']):
    idx = adata.obs['celltype'] == i
    co = np.median(adata.obsm['X_umap'][idx, :], axis=0)
    mean_co[i] = co

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(13,5.5))
sc.pl.umap(adata,  color = ['Study'], legend_loc="right margin", s=30, ax=ax[0], show=False, title="")
sc.pl.umap(adata,  color = ['louvain_rd10'], legend_loc="", s=30, ax=ax[1], show=False, title="")
for i in np.unique(adata.obs['celltype']):
    ax[1].text(mean_co[i][0], mean_co[i][1], i,ha='center', va='center', fontweight='bold', fontsize=13)
plt.tight_layout()
plt.savefig('Rshiny_umap_all.png')

In [ ]:
sc.pl.umap(adata, color=['louvain_rd10', 'celltype'], legend_loc='on data', legend_fontsize='x-small')

In [ ]:
adata.write('./write/all_v1.h5ad')

In [ ]:
adata = sc.read('./write/all_v1.h5ad')

In [ ]:
def add_obs(adata, groups, subgroup, keyName):
    idx = [np.in1d(x, subgroup)[0] for x in adata.obs[groups]]
    cl = np.array(['others']*adata.shape[0])
    cl[idx] = adata.obs[groups][idx]
    adata.obs[keyName] = cl
    adata.obs[keyName] = adata.obs[keyName].astype('category')
    adata.obs[keyName] = adata.obs[keyName].cat.reorder_categories(np.append(np.array(subgroup), 'others'))
    col = np.array([adata.uns[groups+'_colors'][int(x)] for x in subgroup])
    adata.uns[keyName+'_colors'] = np.append(col, '#d3d3d3')

In [ ]:
add_obs(adata, 'louvain_rd10', ['0', '1', '2','3','4','5','6'], 'cEndo')

In [ ]:
sc.pl.umap(adata, color=['cEndo'], ncols=2, wspace=0.5, legend_loc='right margin', legend_fontsize=15, color_map=cmap)

In [ ]:
cls = {
    '0': 'Endo',
    '1': 'Endo', 
    '2': 'Endo', 
    '3': 'Endo',
    '4': 'Endo',
    '6': 'Endo'
}

In [ ]:
adata.obs['celltype'].cat.categories

In [ ]:
adata.uns['celltype_colors']

In [ ]:
adata.obs['Endo_cells'] = [cls[x] if x in cls.keys() else None for x in adata.obs['louvain_rd10']]

In [ ]:
adata.uns['Endo_cells_colors'] = ['#d62728', '#808080']

In [ ]:
sc.pl.umap(adata, color=['Endo_cells'], legend_loc=None, title='Endothelial Cells', save="_celltype_endo.pdf")

In [ ]:
sc.pl.umap(adata, color = "pcna", color_map=cmap)

In [ ]:
study = adata.obs['Study'].cat.categories
fig, ax = plt.subplots(1,3, figsize=(12,4), sharex=True, sharey=True)
for i in range(len(study)):
    sc.pl.umap(adata[adata.obs['Study'] == study[i]], title=study[i], ax = ax[i], show=False, color='celltype', legend_loc='on data', legend_fontsize="xx-small")

In [ ]:
tocol = ['runx1_cit','kdrl_mchr','DBpos']
fig, ax = plt.subplots(1,3, figsize=(12,4), sharex=True, sharey=True)
for i in range(len(tocol)):
    cat = adata.obs[tocol[i]].cat.categories
    col = adata.uns['celltype_colors']
    catCT = adata.obs['celltype'].cat.categories
    adata_sub = adata[adata.obs[tocol[i]] == cat[0]].copy()
    catCTnew = adata_sub.obs['celltype'].cat.categories
    adata_sub.uns['celltype_colors'] = col[np.in1d(np.array(list(catCT)), np.array(list(catCTnew)))]
    sc.pl.umap(adata_sub, color='celltype', s=20,legend_loc='on data', ax=ax[i], show=False, title=tocol[i]+'+',legend_fontsize="xx-small")
    del adata_sub